# Retrieving data from MyAnimeList
This Jupyter Notebook should allow you to get information about the top rated Anime/Manga from MAL via the official API. We're fetching:
- The items' title
- the average user score
- the url to the picture of the item

Simply run all code cells below to get the data

### Imports and entering your Client information
When prompted in the output, please enter your Client-ID and Client-Secret for API access. You get those after creating an ID on [this site](https://myanimelist.net/apiconfig). You will have to do this everytime you re-use this Notebook and get data from MAL.

In [1]:
import json
import requests
import secrets

print('Please enter your Client-ID: ')
CLIENT_ID = input()

print('Please enter your Client-Secret: ')
CLIENT_SECRET = input()

Please enter your Client-ID: 
Please enter your Client-Secret: 


### API Demo
The following cell is a copy from [this](https://gitlab.com/-/snippets/2039434) program. It accesses the API to retrieve user information. This step is just to test, if you can access the API correctly. You can totally skip this cell.

In [10]:
# 1. Generate a new Code Verifier / Code Challenge.
def get_new_code_verifier() -> str:
    token = secrets.token_urlsafe(100)
    return token[:128]


# 2. Print the URL needed to authorise your application.
def print_new_authorisation_url(code_challenge: str):
    global CLIENT_ID

    url = f'https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id={CLIENT_ID}&code_challenge={code_challenge}'
    print(f'Authorise your application by clicking here: {url}\n')


# 3. Once you've authorised your application, you will be redirected to the webpage you've
#    specified in the API panel. The URL will contain a parameter named "code" (the Authorisation
#    Code). You need to feed that code to the application.
def generate_new_token(authorisation_code: str, code_verifier: str) -> dict:
    global CLIENT_ID, CLIENT_SECRET

    url = 'https://myanimelist.net/v1/oauth2/token'
    data = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'code': authorisation_code,
        'code_verifier': code_verifier,
        'grant_type': 'authorization_code'
    }

    response = requests.post(url, data)
    response.raise_for_status()  # Check whether the request contains errors

    token = response.json()
    response.close()
    print('Token generated successfully!')

    with open('token.json', 'w') as file:
        json.dump(token, file, indent = 4)
        print('Token saved in "token.json"')

    return token


# 4. Test the API by requesting your profile information
def print_user_info(access_token: str):
    url = 'https://api.myanimelist.net/v2/users/@me'
    response = requests.get(url, headers = {
        'Authorization': f'Bearer {access_token}'
        })
    
    response.raise_for_status()
    user = response.json()
    response.close()

    print(f"\n>>> Greetings {user['name']}! <<<")
    

# main program
code_verifier = code_challenge = get_new_code_verifier()
print_new_authorisation_url(code_challenge)

authorisation_code = input('Copy-paste the Authorisation Code: ').strip()
token = generate_new_token(authorisation_code, code_verifier)

print_user_info(token['access_token'])

Authorise your application by clicking here: https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id=a14143493714f3b61fc5af33fa325a50&code_challenge=H4GaTCcOGZIPDnmXYaJgs0jyAHTK0IpCZVt_CGGEAWFgZY8c6PkA31aNopyEDDkRdMqJPewZ4Kg7Z9Nn1EJfb2xPnGGGYz7RYgU2-vmoL9_afo3CZvDLv_czodF3aZaY



HTTPError: 400 Client Error: Bad Request for url: https://myanimelist.net/v1/oauth2/token

### Getting information on Anime
We get information for the top 500 rated Anime and dump everything into JSON-format. For some reason, this gets all attributes, even when specifying fields. How do u fix dis?

In [22]:
def get_anime_ranking():
    url = 'https://api.myanimelist.net/v2/anime/ranking?ranking_type=all&limit=500&fields=title,main_picture,alternative_titles,start_date,end_date,mean'
    response = requests.get(url, headers = {
        'X-MAL-CLIENT-ID': f'{CLIENT_ID}'
        })
    
    response.raise_for_status()
    anime_data = response.json()
    
    with open('anime_data.json', 'w') as f:
        json.dump(anime_data, f)
    response.close()

    print("Finished retrieving data and saving it in anime_data.json!")

# main program
get_anime_ranking()

Finished retrieving data and saving it in data.json!


### Getting information on Manga
We do the same as above for the top 500 Manga!

In [3]:
def get_manga_ranking():
    url = 'https://api.myanimelist.net/v2/manga/ranking?ranking_type=all&limit=500&fields=title,main_picture,alternative_titles,start_date,end_date,mean'
    response = requests.get(url, headers = {
        'X-MAL-CLIENT-ID': f'{CLIENT_ID}'
        })
    
    response.raise_for_status()
    manga_data = response.json()
    
    with open('manga_data.json', 'w') as f:
        json.dump(manga_data, f)
    response.close()

    print("Finished retrieving data and saving it in manga_data.json!")

# main program
get_manga_ranking()

Finished retrieving data and saving it in manga_data.json!
